# MODULES


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn import datasets
from statsmodels.distributions import ECDF

sns.set()


___

# POPULATION MEAN INFERENCE
## Central Limit Theorem

Let ${X_{1},\ldots ,X_{n}}$ be a sequence of independent and identically distributed (i.i.d.) random variables drawn from a distribution of expected value $\mu$ and finite variance $\sigma^2$. Let ${\bar {X}}_{n}$ be the sample average: ${\bar {X}}_{n} = ({X_{1} + \ldots + X_{n}}) / n$.

The [Law of Large Numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers) states that the sample mean converges to $\mu$ as the sample size increases.

The [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) states that during this convergence, the difference between the sample mean and its limit $\mu$ approximates the normal distribution with mean 0 and variance $\sigma ^{2}/n$. A very important property of the CLT is that it holds regardless of the distribution of $X_i$ _(see example below)_.

This means that for large samples (typically $n$ greater than 30), the sampling distribution of the sample mean is approximately normal and has the following parameters:

+ mean: $\mu$
+ standard deviation (called **standard error**): $\sigma / \sqrt{n}$


The example below illustrates the validity of the CLT even for distributions that are far from normal, provided the sample is large enough:

![missing](../../img/exp-clt.png)


## CLT and Sample Variance


## Z-Scores

We can use two factors to assess the probability of observing the experimental results under the null hypothesis:
+ The [**Z-score**](https://en.wikipedia.org/wiki/Standard_score) represents the number of standard deviations an observation is from the mean.
+ The sampling distribution of sample statistic is centered around the population parameter and has a standard error linked to the population variance. 

It means that we can calculate the z-score of our sample statistic to calculate its p-value.


## Confidence Interval

As shown in the example, the sample means can take a large range of values; some are quite far from the actual population mean. We usually have only one sample to study the population, with no way of knowing where our sample mean sits in the sampling distribution. What we can do is leverage the CLT to quantify this uncertainty and build a [confidence interval](https://en.wikipedia.org/wiki/Confidence_interval) for the population mean. 

Given a confidence percentage $1 - \alpha$, we can calculate the interval of values inside which $1 - \alpha$ percent of all samples means will fall. A small percentage $\alpha$ of all samples, the ones least representative of the population, will have a sample mean so far from the actual mean that it falls outside of this interval. 

Assuming our sample mean is indeed out of these extreme values, we have according to the CLT: $\mu - z_{\alpha/2} * se < \bar{X} < \mu + z_{1-\alpha/2} * se$. The normal distribution is symmetrical, so $z_{\alpha/2} = - z_{1 - \alpha/2}$. It follows that:

$$\mu \in \bar {X} \pm z_{1-\alpha/2} \frac{\widehat {\sigma }}{\sqrt {n}}$$

Where $z_{1 - \alpha/2}$ is the Z-score above which lay $\alpha/2$ percent of all observations in a standard normal distribution.

As an example, 95% of all sample means are between -2 and +2 standard errors from the sampling distribution mean, which is the population mean $\mu$. It follows that for 95% of all the samples that could have been drawn from the population, the population mean is less than two standard errors away from the sample mean.

_Note: we have no way of knowing if our sample is part of these 95%; for $\alpha =$ 5% of all the possible samples, the confidence interval will not include $\mu$ and the inference will be incorrect. This is why $\alpha$ is called the Type I Error._


## Limits of the CLT

The CLT Confidence intervals **do not works** when either:

+ $\sigma$ is unknown.
+ the sample size $n$ is small.

The [Student’s t distribution](https://en.wikipedia.org/wiki/Student's_t-distribution) is used instead. 

## T-Distribution
### Assumptions of the t-distribution

The sampling distribution of the sample mean has to be roughly normal for the t-distribution to work well. It means that either:

+ the population is normally distributed, even for small samples.
+ the sample is large, regardless of the underlying distribution of data, thanks to the CLT.

If the sample size is very small, we can use normal probability plots to check whether the sample may come from a normal distribution. 

_Note: When the sample size is large (30+ observations), the Student Distribution becomes extremely close to the normal distribution._


### Properties of the t-distribution

Small samples are more likely to underestimate $\sigma$ and have a mean that differs from $\mu$. The t-distribution accounts for this uncertainty with heavier tails compared to a Gaussian: the probability of extreme values becomes comparatively higher. This means its confidence intervals are wider than CLT ones for the same confidence level.


### Mathematical Notions

We have seen that under the assumptions of the CLT:

$$\frac { \bar {X_n} - \mu }{\sigma /\sqrt {n}} \sim N(0, 1)$$

Under the assumptions of the t-distribution, we can substitute the unbiased sample variance $\widehat {\sigma}^2$. In this case, the sampling distribution of the sample mean follows a t-distribution with $n-1$ degrees of freedom ([mathematical proof](https://www.math.arizona.edu/~jwatkins/ttest.pdf)):

$$\frac { \bar {X_n} - \mu }{\widehat {\sigma} /\sqrt {n}} \sim t_{n-1}$$

_Note: the [unbiased](https://dawenl.github.io/files/mle_biased.pdf) variance calculated from a sample of size $n$ uses $n-1$ to average the distances from the mean, in what is called the [Bessel's correction](https://en.wikipedia.org/wiki/Bessel%27s_correction)._

_Note: formally, the t-distribution comes from the division of a normal distribution by a $\chi^2$ distribution. This is the case here: the CLT states that the sampling distribution of the sample mean is asymptotally normal and the sampling distribution of the sample variance is asymptotally  $\chi^2$._


### Confidence intervals

Similarly to what we saw for situations with known variance, the $1 - \alpha$ T Confidence Interval is:

$$\bar{y} \pm T_{\alpha/2, n-1} \times \widehat {\sigma}^2 / \sqrt{n}$$

Where $T_{\alpha/2, n-1}$ is the distance from the mean of the t-distribution with n-1 degrees of freedom above which lay $\alpha/2$ percent of all observations.

### Example for the Exponential distribution

Back to the Exponential Distribution, the figure below shows the experimental sample mean distribution vs T vs Normal for different sample sizes: 2, 5, 10 and 20:
+ the t-distribution gets close to normal even for relatively small sample sizes. 
+ it does not approximate the empirical distribution very well for smaller sample sizes because its assumptions are not met: the exponential distribution is far from normal.

<img class="center-block" src="https://sebastienplat.s3.amazonaws.com/dc954f3e9562d53b7829a2adcd2854ff1490011103173"/>


### Example

We can calculate the confidence interval for the mean petal length of three species of iris, based on their respective samples. The sample size is n=50, which is large enough that we can use the t-distribution.


In [2]:
# load & format iris dataset
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns = iris.feature_names)
iris_df['species'] = [iris.target_names[i] for i in iris.target]

# petal lenghts of each species
setosa_petal_length = iris_df.loc[iris_df['species'] == 'setosa', 'petal length (cm)'].to_numpy()
versicolor_petal_length = iris_df.loc[iris_df['species'] == 'versicolor', 'petal length (cm)'].to_numpy()
virginica_petal_length = iris_df.loc[iris_df['species'] == 'virginica', 'petal length (cm)'].to_numpy()

# sample statistics for iris species
for species, petal_length in zip(['setosa', 'versicolor', 'virginica'], [setosa_petal_length, versicolor_petal_length, virginica_petal_length]):

    # sample stats
    size = len(petal_length)
    mean = np.mean(petal_length)
    var = np.var(petal_length)
    se = np.sqrt(var/size)
    median = np.median(petal_length)

    # CI for pop mean
    tdist = stats.t(df=size-1, loc=mean, scale=se)
    interval = tdist.interval(0.95)
    
    print('{: <10}: median: {:.2f} - mean: {:0.3} - se: {:0.3} - mean 95% CI: [{:0.3f}, {:0.3f}]'.format(species, median, mean, se, interval[0], interval[1]))


setosa    : median: 1.50 - mean: 1.46 - se: 0.0243 - mean 95% CI: [1.413, 1.511]
versicolor: median: 4.35 - mean: 4.26 - se: 0.0658 - mean 95% CI: [4.128, 4.392]
virginica : median: 5.55 - mean: 5.55 - se: 0.0773 - mean 95% CI: [5.397, 5.707]


![iris-ci](../../img/iris-ci.png)


___

# POPULATION MEDIAN INFERENCE

In cases where the distribution of the variable of interest is not unimodal and symmetrical, it is more useful to focus on percentiles (median and IQR). We make no assumptions about the shape of the distribution so 

